In [1]:
import hail as hl

In [2]:
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://localhost:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /vol/bmd/yanyul/GitHub/ptrs-ukb/notebook/hail-20191219-2129-0.2.28-61941242c15d.log


In [11]:
import sys 
sys.path.insert(0, '../code/')
from importlib import reload 
import my_hail_helper as myhelper

In [3]:
gwas_out = hl.read_table('/vol/bmd/yanyul/UKB/gwas_on_subset/from_gcp/all_chrs_bottom.ht')

In [4]:
gwas_out.describe()

----------------------------------------
Global fields:
    'phenotypes': array<array<str>> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'varid': str 
    'rsid': str 
    'n': array<int32> 
    'sum_x': array<float64> 
    'y_transpose_x': array<array<float64>> 
    'beta': array<array<float64>> 
    'standard_error': array<array<float64>> 
    't_stat': array<array<float64>> 
    'p_value': array<array<float64>> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [5]:
gwas_out = gwas_out.annotate( 
    variant = hl.delimit(
        hl.array([
            gwas_out['locus'].contig,
            hl.str(gwas_out['locus'].position),
            gwas_out['alleles'][0],
            gwas_out['alleles'][1]
        ]), 
    delimiter = ':')
)

In [6]:
gwas_out = gwas_out.key_by('variant')
## Hey, this repartition is important
## in the sense that it avoids the unnecessary and repeated sorting caused by key_by
gwas_out = gwas_out.repartition(40)
gwas_out = gwas_out.cache()

2019-12-19 21:30:18 Hail: INFO: Ordering unsorted dataset with network shuffle


In [7]:
phenotypes = gwas_out['phenotypes'].collect()[0]

In [8]:
phenotypes

[['subset13_x_height',
  'subset13_x_dbp',
  'subset13_x_sbp',
  'subset13_x_bmi',
  'subset13_x_wbc',
  'subset13_x_rbc',
  'subset13_x_hb',
  'subset13_x_ht',
  'subset13_x_mcv',
  'subset13_x_mch',
  'subset13_x_mchc',
  'subset13_x_platelet',
  'subset13_x_lymphocyte',
  'subset13_x_monocyte',
  'subset13_x_neutrophil',
  'subset13_x_eosinophil',
  'subset13_x_basophil'],
 ['subset14_x_height',
  'subset14_x_dbp',
  'subset14_x_sbp',
  'subset14_x_bmi',
  'subset14_x_wbc',
  'subset14_x_rbc',
  'subset14_x_hb',
  'subset14_x_ht',
  'subset14_x_mcv',
  'subset14_x_mch',
  'subset14_x_mchc',
  'subset14_x_platelet',
  'subset14_x_lymphocyte',
  'subset14_x_monocyte',
  'subset14_x_neutrophil',
  'subset14_x_eosinophil',
  'subset14_x_basophil'],
 ['subset15_x_height',
  'subset15_x_dbp',
  'subset15_x_sbp',
  'subset15_x_bmi',
  'subset15_x_wbc',
  'subset15_x_rbc',
  'subset15_x_hb',
  'subset15_x_ht',
  'subset15_x_mcv',
  'subset15_x_mch',
  'subset15_x_mchc',
  'subset15_x_platel

In [12]:
phenotypes = gwas_out['phenotypes'].collect()[0]
for i, subset in enumerate(phenotypes):
    for j, trait in enumerate(subset):
        ht_export = myhelper.gwas_formater_from_neale_lab(gwas_out, i, j)
        ht_export.export(f'test_output_with_variant_qc/gwas_test_{trait}.tsv')

2019-12-19 22:11:39 Hail: INFO: merging 40 files totalling 1.3G...
2019-12-19 22:11:53 Hail: INFO: while writing:
    test_output_with_variant_qc/gwas_test_subset13_x_height.tsv
  merge time: 13.738s
2019-12-19 22:12:50 Hail: INFO: merging 40 files totalling 1.3G...
2019-12-19 22:12:56 Hail: INFO: while writing:
    test_output_with_variant_qc/gwas_test_subset13_x_dbp.tsv
  merge time: 5.465s
2019-12-19 22:13:52 Hail: INFO: merging 40 files totalling 1.3G...
2019-12-19 22:14:01 Hail: INFO: while writing:
    test_output_with_variant_qc/gwas_test_subset13_x_sbp.tsv
  merge time: 8.444s
2019-12-19 22:15:01 Hail: INFO: merging 40 files totalling 1.3G...
2019-12-19 22:15:07 Hail: INFO: while writing:
    test_output_with_variant_qc/gwas_test_subset13_x_bmi.tsv
  merge time: 5.295s
2019-12-19 22:15:59 Hail: INFO: merging 40 files totalling 1.3G...
2019-12-19 22:16:08 Hail: INFO: while writing:
    test_output_with_variant_qc/gwas_test_subset13_x_wbc.tsv
  merge time: 9.058s
2019-12-19 22:17

KeyboardInterrupt: 